In [ ]:
import pandas as pd
import ray
from ax.service.ax_client import AxClient
from ax.plot.contour import interact_contour, plot_contour
from ax.plot.slice import plot_slice
from ax.utils.notebook.plotting import render, init_notebook_plotting
from ax.service.utils.best_point import get_best_from_model_predictions, get_best_raw_objective_point
from ray import tune
from ray.tune import track
from ray.tune.suggest.ax import AxSearch


from synthesized import HighDimSynthesizer


def train_evaluate(parameterization):
    with HighDimSynthesizer(df=data, **parameterization) as synthesizer:
        synthesizer.learn(data, num_iterations=None)
        
        data_ = synthesizer.preprocess(data.sample(loss_sample_size))
        feed_dict = synthesizer.get_data_feed_dict(data_)
        losses = synthesizer.get_losses(data=feed_dict)

        track.log(
            mean_loss=losses['total-loss'].numpy().item(),
            reconstruction_loss = losses['reconstruction-loss'].numpy().item(),
            kl_loss = losses['kl-loss'].numpy().item()
        )

In [ ]:
init_notebook_plotting()

In [ ]:
ray.init(address='auto', redis_password='5241590000000000')

In [ ]:
loss_sample_size = 50_000
data = pd.read_csv('data/credit_with_categoricals.csv')
data = data.dropna()
loss_sample_size = min(loss_sample_size, len(data))

ax = AxClient(enforce_sequential_optimization=False)
ax.create_experiment(
    name="capacity_tuning",
    parameters=[
        {"name": "capacity", "type": "range", "bounds": [8, 128]},
        {"name": "latent_size", "type": "range", "bounds": [8, 128]},
        {"name": "num_layers", "type": "range", "bounds": [1, 4]},
        {"name": "residual_depths", "type": "range", "bounds": [2, 6]},
        {"name": "learning_rate", "type": "range", "bounds": [1e-5, 1e-1], "log_scale": True},
        {"name": "max_training_time", "type": "fixed", "value": 120.0}
    ],
    objective_name="mean_loss",
    minimize=True
)

In [ ]:
analysis = tune.run(
    train_evaluate,
    num_samples=40,
    search_alg=AxSearch(ax, max_concurrent=20),  # Note that the argument here is the `AxClient`.
    verbose=0,  # Set this level to 1 to see status updates and to 2 to also see trial results.
    # To use GPU, specify: resources_per_trial={"gpu": 1}.
    resources_per_trial={"cpu": 2},
    max_failures=3
)

In [ ]:
# Gets the best parameters from comparing all trials
params, mean_value = get_best_raw_objective_point(ax.experiment)
print("Best Trial Params:")
print(params, mean_value)

# Gets the parameters by predicting with the bayesian model
params, (mean_value, variance) = get_best_from_model_predictions(ax.experiment)
print("Estimated Best Params:")
print(params, mean_value, variance)



In [ ]:
render(ax.get_feature_importances())

In [ ]:
for param in params:
    try:
        render(plot_slice(ax.generation_strategy.model, param, 'mean_loss'))
    except ValueError:
        pass

In [ ]:
render(interact_contour(ax.generation_strategy.model, metric_name='mean_loss'))

In [ ]:
render(ax.get_optimization_trace())

In [ ]:
ax.get_trials_data_frame()